<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.64
    Uninstalling yfinance-0.2.64:
      Successfully uninstalled yfinance-0.2.64
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-03 15:22:16
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.36 C
-------------------
Today PnL: 15.28 K (0.11%)
Current PnL: -13.41 L (-9.34%)
CY Booked + Current PnL: -6.67 L (-4.65%)
-------------------
Total profit:  4.63 L
Total loss:  -18.04 L
-------------------
Total Booked + Current PnL: 20.31 L (17.43%)
Total Booked PnL: 33.72 L (28.93%)
Curr Year Booked PnL: 6.74 L (4.94%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.08 C
Est FTT PnL: 71.84 L (52.64%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 10.75%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.00,52.86,M-SC,90.62,132543.0,-9936.0,10550.0,-1.62,-6.97,7.96,0.44,243.0,-0.94,0.97,56.44,OX40N,NTT,BANKS
5,APOLLOHOSP,6904.43,8285.00,-16.74,H-LC,88.54,113468.0,9902.0,10802.0,1.66,9.56,9.52,20.00,30.0,0.92,0.83,24.98,X40N,BTT,HEALTHCARE
3,ADANIPORTS,1103.69,1583.00,1.31,M-LC,38.54,101566.0,23204.0,10827.0,-0.93,29.61,10.66,43.43,77.0,2.14,0.74,36.50,XY24,NTT,MISC
88,UNIONBANK,123.87,163.00,0.15,M-LC,34.38,174404.0,33564.0,10935.0,-0.81,23.83,6.27,31.59,89.0,3.07,1.28,51.50,XY24,NTT,BANKS
39,HINDALCO,651.95,761.55,-19.90,H-LC,82.29,110728.0,6416.0,11117.0,-0.90,6.15,10.04,16.81,27.0,0.58,0.81,23.18,X5K,ATH,METALS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-29.63,L-SC,100.00,141464.0,-378.0,56953.0,-0.64,-0.27,40.26,39.88,259.0,-0.01,1.04,43.36,XY24,NTT,POWER
38,HEROMOTOCO,4311.81,6039.03,-6.19,H-MC,46.88,151095.0,182.0,60272.0,1.73,0.12,39.89,40.06,102.0,0.00,1.11,23.59,AR,ATH,AUTO


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,HONAUT,42646.15,58357.33,-14.25,X-MC,69.79,123720.0,-4218.0,51356.0,6.25,-3.30,41.51,36.84,56.0,-0.08,0.91,26.66,X40N,ATH,ELECTRICAL
15,BLUESTARCO,1646.70,2337.55,-1.90,H-SC,97.92,184100.0,19430.0,49652.0,4.83,11.80,26.97,41.95,121.0,0.39,1.35,20.15,X40N,ATH,AC
50,JCHAC,2282.24,2282.00,20639.13,M-SC,66.67,101360.0,-26445.0,26435.0,3.77,-20.69,26.08,-0.01,222.0,-1.00,0.74,12.50,OX40N,NTT,AC
16,BLUSPRING,226.45,84.55,NaN,NaN,6.25,17246.0,-32347.0,1271.0,2.70,-65.22,7.37,-62.66,NaN,-25.45,0.13,15.66,XY24,ATH,MISC
93,VOLTAS,1278.28,1918.49,-24.80,H-MC,89.58,204780.0,13038.0,82997.0,2.43,6.80,40.53,50.08,100.0,0.16,1.50,14.06,XY25,ATH,AC


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ICICIPRULI,600.83,790.0,-10.61,H-MC,29.17,144414.0,8626.0,34125.0,-3.01,6.35,23.63,31.48,119.0,0.25,1.06,19.24,X40,ATH,INSURANCE
47,INDUSINDBK,1354.50,1800.0,80.59,L-MC,75.00,48886.0,-28320.0,53716.0,-2.44,-36.68,109.88,32.89,253.0,-0.53,0.36,34.63,XR,NTT,BANKS
11,BANDHANBNK,214.25,400.0,-28.38,H-SC,78.65,213649.0,-34881.0,250354.0,-1.97,-14.03,117.18,86.70,129.0,-0.14,1.57,41.34,XY24,NTT,BANKS
92,VIPIND,488.80,489.0,-211.61,H-SC,95.83,80535.0,-14781.0,14818.0,-1.67,-15.51,18.40,0.04,154.0,-1.00,0.59,58.85,OX40N,NTT,MISC
87,UJJIVANSFB,52.77,53.0,52.86,M-SC,90.62,132543.0,-9936.0,10550.0,-1.62,-6.97,7.96,0.44,243.0,-0.94,0.97,56.44,OX40N,NTT,BANKS


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,52.86,M-SC,90.62,132543.0,-9936.0,10550.0,-1.62,-6.97,7.96,0.44,243.0,-0.94,0.97,56.44,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-30.51,M-SC,11.46,147978.0,-23163.0,23114.0,2.07,-13.53,15.62,-0.03,189.0,-1.00,1.08,6.36,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,92.71,148899.0,-25660.0,25700.0,0.59,-14.70,17.26,0.02,127.0,-1.00,1.09,29.12,OX40N,NTT,PAINTS
81,TATAELXSI,7332.28,7332.0,-17.14,X-MC,18.75,87038.0,-15614.0,15606.0,0.52,-15.21,17.93,-0.00,58.0,-1.00,0.64,32.31,OX40N,NTT,IT
92,VIPIND,488.80,489.0,-211.61,H-SC,95.83,80535.0,-14781.0,14818.0,-1.67,-15.51,18.40,0.04,154.0,-1.00,0.59,58.85,OX40N,NTT,MISC


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,5.21,47085.0,5830.0,4977.0,-1.44,14.13,10.57,26.19,1.0,1.17,0.35,10.85,XY24,ATH,IT
83,TCS,3794.03,4998.00,-18.70,X-LC,40.62,265216.0,-30718.0,124625.0,-0.67,-10.38,46.99,31.73,1.0,-0.25,1.94,6.16,X40,BTT,IT
48,INFY,1461.46,2275.00,-10.27,X-LC,62.50,288280.0,26679.0,118944.0,0.23,10.20,41.26,55.67,2.0,0.22,2.11,16.93,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-17.36,X-LC,26.04,258866.0,-11481.0,46104.0,0.18,-4.25,17.81,12.81,6.0,-0.25,1.90,8.26,XY25,NTT,FMCG
61,NESTLEIND,2268.60,2755.00,-11.96,X-LC,28.12,279092.0,13666.0,43231.0,-0.11,5.15,15.49,21.44,9.0,0.32,2.05,11.66,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
94,WHIRLPOOL,1167.49,2270.0,-37.42,M-SC,93.75,206197.0,34576.0,127492.0,1.13,20.15,61.83,94.43,198.0,0.27,1.51,52.19,XR,NTT,DURABLES
88,UNIONBANK,123.87,163.0,0.15,M-LC,34.38,174404.0,33564.0,10935.0,-0.81,23.83,6.27,31.59,89.0,3.07,1.28,51.50,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,1.31,M-LC,38.54,101566.0,23204.0,10827.0,-0.93,29.61,10.66,43.43,77.0,2.14,0.74,36.50,XY24,NTT,MISC
32,ENRIN,1377.95,2975.9,NaN,NaN,71.88,73000.0,38551.0,1394.0,-1.05,111.91,1.91,115.97,NaN,27.65,0.54,11.63,AR,ATH,ELECTRICAL


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,96.05,M-MC,60.42,235053.0,10091.0,157556.0,-0.71,4.49,67.03,74.53,185.0,0.06,1.72,36.47,XY24,BTT,STEEL
54,KPIGREEN,497.21,732.16,26.30,H-SC,67.71,129289.0,3992.0,55219.0,-0.34,3.19,42.71,47.25,139.0,0.07,0.95,61.75,MH,ATH,POWER
20,CAMS,3643.00,5250.99,5.43,H-SC,43.75,118468.0,16464.0,28563.0,-0.29,16.14,24.11,44.14,124.0,0.58,0.87,35.00,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,1.31,M-LC,38.54,101566.0,23204.0,10827.0,-0.93,29.61,10.66,43.43,77.0,2.14,0.74,36.50,XY24,NTT,MISC
88,UNIONBANK,123.87,163.00,0.15,M-LC,34.38,174404.0,33564.0,10935.0,-0.81,23.83,6.27,31.59,89.0,3.07,1.28,51.50,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.90,NaN,NaN,71.88,73000.0,38551.0,1394.0,-1.05,111.91,1.91,115.97,NaN,27.65,0.54,11.63,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-37.42,M-SC,93.75,206197.0,34576.0,127492.0,1.13,20.15,61.83,94.43,198.0,0.27,1.51,52.19,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-3.31,M-LC,78.65,165385.0,14440.0,95014.0,-0.09,9.57,57.45,72.51,69.0,0.15,1.21,13.92,XR,NTT,IT
45,INDIAMART,2327.09,4871.06,-47.52,H-SC,86.46,136502.0,13166.0,121664.0,0.23,10.68,89.13,109.32,122.0,0.11,1.00,34.85,AR,ATH,MISC
54,KPIGREEN,497.21,732.16,26.30,H-SC,67.71,129289.0,3992.0,55219.0,-0.34,3.19,42.71,47.25,139.0,0.07,0.95,61.75,MH,ATH,POWER


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,96.88,67622.0,-45927.0,85975.0,-0.85,-40.45,127.14,35.27,261.0,-0.53,0.50,65.51,XR,NTT,HOTELS
58,MASFIN,326.60,402.39,-18.70,H-SC,64.58,92160.0,-5820.0,28560.0,-0.15,-5.94,30.99,23.21,137.0,-0.20,0.68,33.66,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,8.27,X-SC,98.96,100715.0,-248.0,94783.0,-0.19,-0.25,94.11,93.63,64.0,-0.00,0.74,50.01,SR,ATH,CHEMICALS
3,ADANIPORTS,1103.69,1583.00,1.31,M-LC,38.54,101566.0,23204.0,10827.0,-0.93,29.61,10.66,43.43,77.0,2.14,0.74,36.50,XY24,NTT,MISC
90,VALIANTORG,512.64,838.00,-355.15,H-SC,3.12,108576.0,-24710.0,109303.0,0.30,-18.54,100.67,63.47,148.0,-0.23,0.80,82.00,XR,NTT,CHEMICALS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-29.63,L-SC,100.00,141464.0,-378.0,56953.0,-0.64,-0.27,40.26,39.88,259.0,-0.01,1.04,43.36,XY24,NTT,POWER
4,ALKYLAMINE,2347.98,4546.37,8.27,X-SC,98.96,100715.0,-248.0,94783.0,-0.19,-0.25,94.11,93.63,64.0,-0.00,0.74,50.01,SR,ATH,CHEMICALS
15,BLUESTARCO,1646.70,2337.55,-1.90,H-SC,97.92,184100.0,19430.0,49652.0,4.83,11.80,26.97,41.95,121.0,0.39,1.35,20.15,X40N,ATH,AC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,96.88,67622.0,-45927.0,85975.0,-0.85,-40.45,127.14,35.27,261.0,-0.53,0.50,65.51,XR,NTT,HOTELS
92,VIPIND,488.80,489.00,-211.61,H-SC,95.83,80535.0,-14781.0,14818.0,-1.67,-15.51,18.40,0.04,154.0,-1.00,0.59,58.85,OX40N,NTT,MISC


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.87
1,25,41.94
2,50,70.45


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.49
LC    33.91
MC    21.59
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.55
X40      12.19
XY25     12.02
X40N     11.90
XR       10.11
AR        8.23
OX40N     8.21
MH        1.70
X5K       1.46
X200      1.41
SR        1.23
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.62
M-SC    16.01
H-LC    14.10
X-LC    12.12
H-MC     9.52
X-MC     7.61
M-LC     6.64
X-SC     4.41
M-MC     4.10
L-SC     2.45
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.05,-1.94,36.04
IT,12.07,-10.22,64.38
BANKS,7.88,-6.05,49.32
FINANCE,7.88,-12.43,53.13
MISC,6.62,-15.11,68.52
PAINTS,5.73,-11.62,33.61
ELECTRICAL,5.54,2.22,43.45
HEALTHCARE,4.57,-3.38,33.58
AUTO,4.34,-14.15,64.34


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2699569.0,27
XR,1079943.0,13
AR,1022026.0,10
X40,655381.0,9
X40N,540657.0,11
XY25,481329.0,8
OX40N,308118.0,11
SR,176187.0,2
MH,84502.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1972431.0,20
M-SC,1756914.0,21
H-LC,617046.0,12
H-MC,535460.0,8
X-LC,521496.0,7
X-MC,414813.0,6
M-MC,409988.0,3
X-SC,316047.0,5
M-LC,267718.0,5


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      872424.0      6
M-SC       XY24      812566.0      7
H-SC       AR        521447.0      3
M-SC       XR        332532.0      4
M-MC       XY24      299873.0      2
H-MC       XY24      297680.0      3
X-LC       X40       243569.0      2
H-SC       XR        217651.0      3
           X40N      216105.0      4
H-LC       X40N      199937.0      4
M-SC       AR        181954.0      2
           OX40N     162386.0      5
M-LC       XY24      157822.0      3
H-LC       X40       156289.0      3
X-MC       X40       149447.0      2
L-SC       XR        147696.0      2
X-SC       XR        123219.0      1
X-LC       XY25      119319.0      3
           AR        116452.0      1
M-SC       XY25      114121.0      1
M-MC       XR        110115.0      1
X-MC       XY25      103367.0      1
           XY24       95037.0      1
M-LC       XR         95014.0      1
X-SC       SR         94783.0      1
H-SC       OX40N      89585.0      3
H-MC       XY25       82997.0      1
M-SC       SR         81404.0      1
H-LC       AR         80714.0      1
M-SC       X40        71951.0      1
H-LC       X200       68546.0      1
           X5K        68098.0      2
H-MC       AR         60272.0      1
L-SC       AR         59793.0      1
X-SC       XY24       57504.0      1
L-SC       XY24       56953.0      1
H-SC       MH         55219.0      1
L-MC       XR         53716.0      1
X-MC       X40N       51356.0      1
L-LC       XY25       46643.0      1
H-LC       XY24       43462.0      1
X-LC       X40N       42156.0      1
X-SC       OX40N      40541.0      2
H-MC       X40        34125.0      1
           X40N       31103.0      1
           MH         29283.0      1
X-MC       OX40N      15606.0      1
M-LC       XY25       14882.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 36.0 seconds
